In [ ]:
# https 协议
!pip install -q git+https://github.com/huggingface/transformers.git
# ssh 协议
!pip install -q git+ssh://git@github.com/huggingface/transformers.git

In [1]:
import transformers

In [2]:
transformers.__version__

'4.30.0.dev0'

## basics

- hf（huggingface）中使用 llama
- llama => alpaca
- lora on alpaca
- inference：推理
    - alpaca 标准 prompt 格式

## load model/tokenizer

- https://github.com/tloen/alpaca-lora/blob/main/generate.py

In [3]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

In [ ]:
# import os
# os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
# os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

### load model

In [4]:
model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/whaow/anaconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/whaow/anaconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/whaow/anaconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): Llama

In [7]:
for i, para in enumerate(model.named_parameters()):
#     print(f'{i}, {para[0]}\t {para[1].device} \t{para[1].dtype}')
    print(f'{i}, \t {para[1].device} \t{para[1].dtype}')

0, 	 cuda:0 	torch.float16
1, 	 cuda:0 	torch.int8
2, 	 cuda:0 	torch.int8
3, 	 cuda:0 	torch.int8
4, 	 cuda:0 	torch.int8
5, 	 cuda:0 	torch.int8
6, 	 cuda:0 	torch.int8
7, 	 cuda:0 	torch.int8
8, 	 cuda:0 	torch.float16
9, 	 cuda:0 	torch.float16
10, 	 cuda:0 	torch.int8
11, 	 cuda:0 	torch.int8
12, 	 cuda:0 	torch.int8
13, 	 cuda:0 	torch.int8
14, 	 cuda:0 	torch.int8
15, 	 cuda:0 	torch.int8
16, 	 cuda:0 	torch.int8
17, 	 cuda:0 	torch.float16
18, 	 cuda:0 	torch.float16
19, 	 cuda:0 	torch.int8
20, 	 cuda:0 	torch.int8
21, 	 cuda:0 	torch.int8
22, 	 cuda:0 	torch.int8
23, 	 cuda:0 	torch.int8
24, 	 cuda:0 	torch.int8
25, 	 cuda:0 	torch.int8
26, 	 cuda:0 	torch.float16
27, 	 cuda:0 	torch.float16
28, 	 cuda:0 	torch.int8
29, 	 cuda:0 	torch.int8
30, 	 cuda:0 	torch.int8
31, 	 cuda:0 	torch.int8
32, 	 cuda:0 	torch.int8
33, 	 cuda:0 	torch.int8
34, 	 cuda:0 	torch.int8
35, 	 cuda:0 	torch.float16
36, 	 cuda:0 	torch.float16
37, 	 cuda:0 	torch.int8
38, 	 cuda:0 	torch.int8
39, 	 cu

### load tokenizer

In [8]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [9]:
tokenizer

LlamaTokenizer(name_or_path='decapoda-research/llama-7b-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=False)

## with lora

- https://github.com/tloen/alpaca-lora

In [10]:
from peft import PeftModel

In [11]:
model = PeftModel.from_pretrained(model, "tloen/alpaca-lora-7b")

In [12]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear8bitLt(
                in_features=4096, out_features=4096, b

In [13]:
from peft import mapping
from peft.utils import other

print('model_type', model.config.model_type)
print(model.peft_config['default'].target_modules)

#默认的 target module
other.TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

model_type llama
['q_proj', 'k_proj', 'v_proj', 'o_proj']


{'t5': ['q', 'v'],
 'mt5': ['q', 'v'],
 'bart': ['q_proj', 'v_proj'],
 'gpt2': ['c_attn'],
 'bloom': ['query_key_value'],
 'blip-2': ['q', 'v', 'q_proj', 'v_proj'],
 'opt': ['q_proj', 'v_proj'],
 'gptj': ['q_proj', 'v_proj'],
 'gpt_neox': ['query_key_value'],
 'gpt_neo': ['q_proj', 'v_proj'],
 'bert': ['query', 'value'],
 'roberta': ['query', 'value'],
 'xlm-roberta': ['query', 'value'],
 'electra': ['query', 'value'],
 'deberta-v2': ['query_proj', 'value_proj'],
 'deberta': ['in_proj'],
 'layoutlm': ['query', 'value'],
 'llama': ['q_proj', 'v_proj'],
 'chatglm': ['query_key_value']}

## alpaca examples

- https://github.com/tatsu-lab/stanford_alpaca

In [18]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [19]:
generation_config = GenerationConfig(
    temperature=1.5,
    # nucleus sampling
    top_p=0.8,
    num_beams=4,
)

def inference(instruction, input=None):
    prompt = generate_prompt(instruction, input)
#     print(prompt)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Response:", output.split("### Response:")[1].strip())

In [20]:
inference(input("Instruction: "))

Instruction: tell me some jokes.
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
tell me some jokes.

### Response:
Response: Q: Why don't scientists trust atoms?
A: Because they make up everything!

Q: What do you call a bear with no teeth?
A: A gummy bear!

Q: What did the fish say when it swam into a wall?
A: Dam!
